In [469]:
%matplotlib
import matplotlib.pyplot as plt
plt.ion()

Using matplotlib backend: Qt5Agg


### FFT with rotating plate

The setup was conducted as follows:


1.The laser with wavelength $\lambda$ = 660 nm was calibrated to hit the polarimeter at the end of the setup centrally.


2.A train consisting of linear polarizer LP2 (Pol.), $\frac{\lambda}{2}$-WP 600 - 2700 nm and
$\frac{\lambda}{4}$-WP 690 - 1200 nm was assembled and placed into the laser beam.


3.By rotating the WPs of the train horizontally polarized light was set.

The angles of the optical elements in the train in the order set were $\Theta_{1}=49^{\circ}$
, $\Theta_{2}=227^{\circ}$ and $\Theta_{3}=???^{\circ}$

, because the last plate holder had no markings.

The parameters of the horizontally polarized light were: S1=1.000 ; S2=-0.002 ; S3=-0.004 ; DOP=103.9 % and power=-19.61 dBm

Annotation: The angle of LP2 (Pol.) in the train could have been set that maximum intensity is transmitted, but wasn't.


4.In order to find the angle $\alpha$
of the x-axis following procedure was used. Another linear polarizer LP3, which is here also referred to as P1, was inserted after the train and rotated until maximum intensity was measured by the polarimeter.This was the case for $\alpha_{1}=131^{\circ}$

The measured parameters were:

S1=0.999 ; S2=0.001 ; S3=0.041 ; DOP=103.7 % and power=-21.3 dBm


5.Another linear polarizer LP1, which is here also referred as P2, was inserted behind P1 just in front of the polarimeter and rotated until maximum intensity was measured by the polarimeter.This was the case for $\alpha_{2}=336^{\circ}$

The measured parameters were:

S1=1.000 ; S2=0.002 ; S3=0.002 ; DOP=103.7 % and power=-21.95 dBm

Now the angle of P2 is set to transmit horizontally polarized light most effectively and therefore this angle is assigned to the x-axis. Accordingly, in relation to the x-axis P2 is now set at $\alpha$=0.
This position is not changed during the measurement


6.All components are removed, except the train, and horizontally polarized light intended for measurement is set.

S1=0.999 ; S2=0.003 ; S3=0.020 ; DOP=103.6 % and power=-19.42 dBm


7.Behind the train we insert the $\frac{\lambda}{4}$
-WP 600 - 2700 nm in the rotating wheel and behind the wheel we insert LP1 as P2 at angle $\alpha$=0


8.The wheel is rotated from 0 to 2 𝜋
and 361 equally distanced data points are recorded

### Self-calibrating polarimetric method

In order to obtain a precise calibration of the parameters alpha_0, beta_0 and delta following the procedure described in the paper by Andreev (2017), we need to do two measurements with horizontally polarized light where in the first one the the angle of P2 is 0° and in the second one 45°. 

### Read in data for setup like in paper

In [470]:

import numpy as np
import matplotlib.pyplot as plt

from sympy import Symbol, cos, sin, pi, sqrt, symbols
from sympy.matrices import *
from sympy.utilities.lambdify import lambdify
from sympy.interactive.printing import init_printing
from sympy.interactive import printing
printing.init_printing(use_latex=True)

from scipy.optimize import curve_fit, minimize
from scipy import fftpack
import os
from os.path import join, getsize

import matplotlib as mpl
mpl.use('pdf')
# Define some formatting parameters
#Direct input 
plt.rcParams['text.latex.preamble']=[r"\usepackage{lmodern}"]
#Options
params = {'text.usetex' : True,
          'font.size' : 8,
          'font.family' : 'lmodern',
          'text.latex.unicode': True,
          }
plt.rcParams.update(params) 

plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=8.7)
plt.rc('ytick', labelsize=8.7)
plt.rc('axes', labelsize=8.7)

# width as measured in inkscape
width = 2.809
height = width / 1.618


directories=[]

# Some names for the data files to know which polarization
# 20190515[0]horizontal polarization
# 20190515[1]vertical polarization
# 20190516[0]diagonal polarization
# 20190520[0]antidiagonal polarization
# 20190521[0]left-circular polarization
# 20190521[1]right-circular polarization
# 20190527[0]horizontal polarization at alpha=0
# 20190527[1]horizontal polarization at alpha=45
# 20190528 different other polarizations (names will be added)


for root, dirs, files in os.walk('20190527/'):
    if files != []:
        a=str(root)+str('/')+str(files[0])
        directories.append(a)

for root, dirs, files in os.walk('20190528/'):
    if files != []:
        a=str(root)+str('/')+str(files[0])
        directories.append(a)
# Only horizontally polarized light
directories=directories[:]
print(directories)
data=[]
measurement_point_all=[]
angle_all=[]
s1_all=[]
s2_all=[]
s3_all=[]
dop_all=[]
power_all=[]

for i in directories:
    f=open(i)
    lines=f.readlines()
    for i in range(26):
        lines.pop(0)
    measurement_point=[]
    angle=[]
    s1=[]
    s2=[]
    s3=[]
    dop=[]
    power=[]
    for line in lines:
        measurement_point.append( line.strip().split('\t')[0] )
        angle.append( line.strip().split('\t')[1] )
        s1.append( line.strip().split('\t')[2] )
        s2.append( line.strip().split('\t')[3] )
        s3.append( line.strip().split('\t')[4] )
        dop.append( line.strip().split('\t')[5] )
        power.append( line.strip().split('\t')[6] )
    measurement_point_all.append(measurement_point)
    angle_all.append(angle)
    s1_all.append(s1)
    s2_all.append(s2)
    s3_all.append(s3)
    dop_all.append(dop)
    power_all.append(power)
measurement_point=np.array(measurement_point_all).astype(np.float)  
angle=np.array(angle_all).astype(np.float)
s1=np.array(s1_all).astype(np.float) 
s2=np.array(s2_all).astype(np.float)
s3=np.array(s3_all).astype(np.float) 
dop=np.array(dop_all).astype(np.float)
power=np.array(power_all).astype(np.float)

['20190527/174655_polarimeter_vs_wheel/174655_polarimeter_vs_wheel.dat', '20190527/175544_polarimeter_vs_wheel/175544_polarimeter_vs_wheel.dat', '20190528/142032_polarimeter_vs_wheel/142032_polarimeter_vs_wheel.dat', '20190528/143851_polarimeter_vs_wheel/143851_polarimeter_vs_wheel.dat', '20190528/145841_polarimeter_vs_wheel/145841_polarimeter_vs_wheel.dat', '20190528/151432_polarimeter_vs_wheel/151432_polarimeter_vs_wheel.dat']


In [471]:
plt.figure()
plt.plot(angle[0], power[0])
plt.ylabel('Intensity')
plt.xlabel('Degree °')
plt.title(r'Measurement for $\frac{1}{2}$ WP40')

Text(0.5, 1.0, 'Measurement for $\\frac{1}{2}$ WP40')

### Calculate discrete Fourier series coefficients like in paper
For an uneven number of data points like in our case with N = 2L = 361 we can calculate the discrete Fourier series coefficients according to
$$C_{\omega, k}=\frac{2}{N} \frac{1}{1+\delta_{k0}} \sum_{i=1}^N I_{T_{i}} cos \omega_{k} \beta_{i}$$
$$S_{\omega, k}=\frac{2}{N} \frac{1}{1+\delta_{k0}} \sum_{i=1}^N I_{T_{i}} sin \omega_{k} \beta_{i}$$
with the discrete angular velocities
$$\omega_{k} = \frac{2\pi}{N} \frac{k}{\Delta \beta}$$ the angular positions $$\beta_{i}=(i-1)\Delta \beta$$ and $\Delta \beta$ being the difference in the angle for two neighboring data points.

We need to consider that due to the dependance of the discrete angular velocities on the step size, it can happen that the 2 $\omega$ and 4$\omega$ signals, which correspond to circular and linear polarizations, are not exactly included in our calculations. Therefore, we look for the discrete angular frequencies, which are closest to the requirements.

### Starting Parameters
Calculate the coefficients for $\alpha$=0



In [472]:
# Calculate discrete Fourier coefficients for an uneven number of data points N=2L+1
names=['H-light', 'L-light', 'E-light']
# With the intensity variable the measurement for the calculations is chosen
intensity=power[0]

L=180
N=2*L
index=np.linspace(0,359,360)*(np.pi/180)


omega_k=[]
# Define step size
delta_beta=(index[1]-index[0])
#print(index)
for i in np.arange(L):
    omega_k.append(2*np.pi*i/(N*delta_beta))
omega_k=np.array(omega_k)
# Calculate Fourier series coefficients
C_omega_k=[]
S_omega_k=[]

# Sum in definition of C_omega_k with cos terms

def sum_cos(x):
    ''' Calculate formula from above with variable x=omega_{k}'''
    sum_cos=0
    j=0
    for i in index:
        sum_cos=sum_cos+ intensity[j]*np.cos(x*i)
        j+=1
    return sum_cos
a=sum_cos(omega_k)

# Sum in definition of S_omega_k with sin terms
def sum_sin(x):
    '''Same with sin'''
    sum_sin=0
    j=0
    for i in index:
        sum_sin+= intensity[j]*np.sin(x*i)
        j+=1
    return sum_sin

#print(sum_sin(omega_k))
def fourier_coefficients(x):
    '''Calculate complete Fourier coefficients'''
    C_omega_k=(2/N)* sum_cos(x)
    S_omega_k=(2/N)* sum_sin(x)
    return C_omega_k, S_omega_k

sigma_I=0.134*10**(-6)

def sum_cos_inten_err(x):
    ''' Calculate error on the cosine sum'''
    sum_cos_error=0
    for i in index:
        sum_cos_error=sum_cos_error+ (np.cos(x*i))**2
        
    return sum_cos_error

def sum_sin_inten_err(x):
    ''' Same with sin'''
    sum_sin_error=0
    for i in index:
        sum_sin_error=sum_sin_error+ (np.sin(x*i))**2
        
    return sum_sin_error

def intensity_error(x):
    ''' Calculate error on intensity with errors from the cosine and sine sums'''
    C_omega_k_inten_error=np.sqrt(sum_cos_inten_err(x)*(2*sigma_I/N)**2 )
    S_omega_k_inten_error=np.sqrt(sum_sin_inten_err(x)*(2*sigma_I/N)**2 )
    return C_omega_k_inten_error, S_omega_k_inten_error

# Defined error on the angle beta
sigma_beta=0.3*np.pi/180

def sum_cos_beta_err(x):
    ''' Calculate error on the cosine sum from beta error'''
    sum_cos=0
    j=0
    for i in index:
        sum_cos=sum_cos+ (intensity[j]*(np.sin(x*i)))**2
        j+=1
    return sum_cos

def sum_sin_beta_err(x):
    ''' Calculate error on the sine sum from beta error'''
    sum_sin=0
    j=0
    for i in index:
        sum_sin=sum_sin+ (intensity[j]*(x*np.cos(x*i)))**2
        j+=1
    return sum_sin

def beta_error(x):
    ''' Calculate error on the fourier coefficients from beta error '''
    C_omega_k_beta_error=np.sqrt(sum_cos_beta_err(x)*(2*sigma_beta/N)**2)
    S_omega_k_beta_error=np.sqrt(sum_sin_beta_err(x)*(2*sigma_beta/N)**2)
    return C_omega_k_beta_error, S_omega_k_beta_error



# Loop to get all Fourier coefficients
Cs=[]
indices=[]
Ss=[]

Cs_inten_err=[]
Ss_inten_err=[]

Cs_beta_err=[]
Ss_beta_err=[]


# Calculate only first 11 Fourier coefficients

# Find indices, where omega_k is closest to desired integer values

for i in np.arange(11):
    indices.append(np.where(  np.absolute(omega_k-i)==np.amin(np.absolute(omega_k-i)) )[0][0] )

for i in np.arange(11):
    Cs.append(fourier_coefficients(omega_k)[0][indices[i]])
    Ss.append(fourier_coefficients(omega_k)[1][indices[i]])
    Cs_inten_err.append(intensity_error(omega_k)[0][indices[i]])
    Ss_inten_err.append(intensity_error(omega_k)[1][indices[i]])
    Cs_beta_err.append(beta_error(omega_k)[0][indices[i]])
    Ss_beta_err.append(beta_error(omega_k)[1][indices[i]])
    
Cs[0]=Cs[0]*0.5
#Calculating Fourier coefficients, which correspond to omega_k=2, 4
# Same method as above, did it just to check if the method is right
# Find index of entry of omega_k, which is closest to omega_k = 0,2 or 4
index_0,=np.where( omega_k==np.amin(omega_k) )
index_2,=np.where(  np.absolute(omega_k-2)==np.amin(np.absolute(omega_k-2)) )
index_4,=np.where(  np.absolute(omega_k-4)==np.amin(np.absolute(omega_k-4)) )


#print('Cs=', Cs)
#print(omega_k)
# Calculate Fourier coefficients for omega_k = 2,4
# Factor 0.5 for omega_k=0 to account for special case of fourier series coefficient formula
C0=fourier_coefficients(omega_k)[0][index_0[0]]*0.5
C2=fourier_coefficients(omega_k)[0][index_2[0]]
C4=fourier_coefficients(omega_k)[0][index_4[0]]
S2=fourier_coefficients(omega_k)[1][index_2[0]]
S4=fourier_coefficients(omega_k)[1][index_4[0]]

C0_inten_err=intensity_error(omega_k)[0][index_0[0]]*0.5
C2_inten_err=intensity_error(omega_k)[0][index_2[0]]
C4_inten_err=intensity_error(omega_k)[0][index_4[0]]
S2_inten_err=intensity_error(omega_k)[1][index_2[0]]
S4_inten_err=intensity_error(omega_k)[1][index_4[0]]

C0_beta_err=beta_error(omega_k)[0][index_0[0]]*0.5
C2_beta_err=beta_error(omega_k)[0][index_2[0]]
C4_beta_err=beta_error(omega_k)[0][index_4[0]]
S2_beta_err=beta_error(omega_k)[1][index_2[0]]
S4_beta_err=beta_error(omega_k)[1][index_4[0]]

# Total error on Fourier coefficients
sigma_C0=np.sqrt(C0_inten_err**2 + C0_beta_err**2  )
sigma_C2=np.sqrt(C2_inten_err**2 + C2_beta_err**2  )
sigma_C4=np.sqrt(C4_inten_err**2 + C4_beta_err**2  )
sigma_S2=np.sqrt(S2_inten_err**2 + S2_beta_err**2  )
sigma_S4=np.sqrt(S4_inten_err**2 + S4_beta_err**2  )


[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111.
 112. 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125.
 126. 127. 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139.
 140. 141. 142. 143. 144. 145. 146. 147. 148. 149. 150. 151. 152. 153.
 154. 155. 156. 157. 158. 159. 160. 161. 162. 163. 164. 165. 166. 167.
 168. 169. 170. 171. 172. 173. 174. 175. 176. 177. 178. 179.]


We need to regard only the 0 $\omega$, 2$\omega$ and 4$\omega$ signals, as these are the only ones contributing to the resulting intensity. As expected for horizontally polarized light the coefficients C2 and S2, which correspond to circular polarized light, are suppressed

In [473]:
print("C0=", Cs[0])
print("C2=", Cs[2]) 
print("C4=", Cs[4])
print("S2=", Ss[2])
print("S4=", Ss[4])

print("C0_inten_err=", Cs_inten_err[0])
print("C2_inten_err=", Cs_inten_err[2]) 
print("C4_inten_err=", Cs_inten_err[4])
print("S2_inten_err=", Ss_inten_err[2])
print("S4_inten_err=", Ss_inten_err[4])

print("C0_beta_err=", Cs_beta_err[0])
print("C2_beta_err=", Cs_beta_err[2]) 
print("C4_beta_err=", Cs_beta_err[4])
print("S2_beta_err=", Ss_beta_err[2])
print("S4_beta_err=", Ss_beta_err[4])


print("sigma_C0=", sigma_C0)
print("sigma_C2=", sigma_C2)
print("sigma_C4=", sigma_C4)
print("sigma_S2=", sigma_S2)
print("sigma_S4=", sigma_S4)

C0= 1.0684267687742498e-05
C2= -5.445373460213112e-07
C4= -1.294584376906714e-06
S2= -5.034491116498676e-07
S4= -3.044785338841824e-06
C0_inten_err= 1.4124840215418762e-08
C2_inten_err= 9.987770299499064e-09
C4_inten_err= 9.987770299499062e-09
S2_inten_err= 9.98777029949906e-09
S4_inten_err= 9.987770299499062e-09
C0_beta_err= 0.0
C2_beta_err= 4.534551324006189e-09
C4_beta_err= 4.342865510328021e-09
S2_beta_err= 8.083180998419888e-09
S4_beta_err= 1.6987585411731115e-08
sigma_C0= 7.062420107709381e-09
sigma_C2= 1.0968943033200688e-08
sigma_C4= 1.0891098952647169e-08
sigma_S2= 1.2848866510660459e-08
sigma_S4= 1.9706182118219028e-08


### Calculate coefficients for $\alpha$=45°

In [474]:
# Calculate discrete Fourier coefficients for an uneven number of data points N=2L+1
names=['H-light', 'L-light', 'E-light']
#%%
intensity=power[1]

L=180
N=2*L
index=np.linspace(0,359,360)*(np.pi/180)


omega_k=[]
# Define step size
delta_beta=(index[1]-index[0])

for i in np.arange(L):
    omega_k.append(2*np.pi*i/(N*delta_beta))
omega_k=np.array(omega_k)
# Calculate Fourier series coefficients
C_omega_k=[]
S_omega_k=[]

# Sum in definition of C_omega_k with cos terms

def sum_cos(x):
    ''' Calculate formula from above with variable x=omega_{k}'''
    sum_cos=0
    j=0
    for i in index:
        sum_cos=sum_cos+ intensity[j]*np.cos(x*i)
        j+=1
    return sum_cos
a=sum_cos(omega_k)

# Sum in definition of S_omega_k with sin terms
def sum_sin(x):
    '''Same with sin'''
    sum_sin=0
    j=0
    for i in index:
        sum_sin+= intensity[j]*np.sin(x*i)
        j+=1
    return sum_sin

#print(sum_sin(omega_k))
def fourier_coefficients(x):
    '''Calculate complete Fourier coefficients'''
    C_omega_k=(2/N)* sum_cos(x)
    S_omega_k=(2/N)* sum_sin(x)
    return C_omega_k, S_omega_k

sigma_I=0.134*10**(-6)

def sum_cos_inten_err(x):
    ''' Calculate error on the cosine sum'''
    sum_cos_error=0
    for i in index:
        sum_cos_error=sum_cos_error+ (np.cos(x*i))**2
        
    return sum_cos_error

def sum_sin_inten_err(x):
    ''' Calculate error on the sin sum'''
    sum_sin_error=0
    for i in index:
        sum_sin_error=sum_sin_error+ (np.sin(x*i))**2
        
    return sum_sin_error

def intensity_error(x):
    ''' Calculate error on intensity with errors from the cosine and sine sums'''
    C_omega_k_inten_error=np.sqrt(sum_cos_inten_err(x)*(2*sigma_I/N)**2 )
    S_omega_k_inten_error=np.sqrt(sum_sin_inten_err(x)*(2*sigma_I/N)**2 )
    return C_omega_k_inten_error, S_omega_k_inten_error

sigma_beta=0.3*np.pi/180

def sum_cos_beta_err(x):
    ''' Calculate error on the cosine sum from beta error'''
    sum_cos=0
    j=0
    for i in index:
        sum_cos=sum_cos+ (intensity[j]*(np.sin(x*i)))**2
        j+=1
    return sum_cos

def sum_sin_beta_err(x):
    ''' Calculate error on the sine sum from beta error'''
    sum_sin=0
    j=0
    for i in index:
        sum_sin=sum_sin+ (intensity[j]*(x*np.cos(x*i)))**2
        j+=1
    return sum_sin

def beta_error(x):
    ''' Calculate error on the fourier coefficients from beta error '''
    C_omega_k_beta_error=np.sqrt(sum_cos_beta_err(x)*(2*sigma_beta/N)**2)
    S_omega_k_beta_error=np.sqrt(sum_sin_beta_err(x)*(2*sigma_beta/N)**2)
    return C_omega_k_beta_error, S_omega_k_beta_error

# Loop to get all Fourier coefficients
Cs_prime=[]
indices_prime=[]
Ss_prime=[]

Cs_inten_err_prime=[]
Ss_inten_err_prime=[]

Cs_beta_err_prime=[]
Ss_beta_err_prime=[]
# Calculate only first 11 Fourier coefficients

# Find indices, where omega_k is closest to desired integer values

for i in np.arange(11):
    indices_prime.append(np.where(  np.absolute(omega_k-i)==np.amin(np.absolute(omega_k-i)) )[0][0] )

for i in np.arange(11):
    Cs_prime.append(fourier_coefficients(omega_k)[0][indices_prime[i]])
    Ss_prime.append(fourier_coefficients(omega_k)[1][indices_prime[i]])
    Cs_inten_err_prime.append(intensity_error(omega_k)[0][indices[i]])
    Ss_inten_err_prime.append(intensity_error(omega_k)[1][indices[i]])
    Cs_beta_err_prime.append(beta_error(omega_k)[0][indices[i]])
    Ss_beta_err_prime.append(beta_error(omega_k)[1][indices[i]])
Cs_prime[0]=Cs_prime[0]*0.5
#Calculating Fourier coefficients, which correspond to omega_k=2, 4
# Same method as above, did it just to check if the method is right
# Find index of entry of omega_k, which is closest to omega_k = 0,2 or 4
index_0,=np.where( omega_k==np.amin(omega_k) )
index_2,=np.where(  np.absolute(omega_k-2)==np.amin(np.absolute(omega_k-2)) )
index_4,=np.where(  np.absolute(omega_k-4)==np.amin(np.absolute(omega_k-4)) )


# Calculate Fourier coefficients for omega_k = 2,4
# Factor 0.5 for omega_k=0 to account for special case of fourier series coefficient formula
C0_prime=fourier_coefficients(omega_k)[0][index_0[0]]*0.5
C2_prime=fourier_coefficients(omega_k)[0][index_2[0]]
C4_prime=fourier_coefficients(omega_k)[0][index_4[0]]
S2_prime=fourier_coefficients(omega_k)[1][index_2[0]]
S4_prime=fourier_coefficients(omega_k)[1][index_4[0]]

C0_inten_err_prime=intensity_error(omega_k)[0][index_0[0]]*0.5
C2_inten_err_prime=intensity_error(omega_k)[0][index_2[0]]
C4_inten_err_prime=intensity_error(omega_k)[0][index_4[0]]
S2_inten_err_prime=intensity_error(omega_k)[1][index_2[0]]
S4_inten_err_prime=intensity_error(omega_k)[1][index_4[0]]

C0_beta_err_prime=beta_error(omega_k)[0][index_0[0]]*0.5
C2_beta_err_prime=beta_error(omega_k)[0][index_2[0]]
C4_beta_err_prime=beta_error(omega_k)[0][index_4[0]]
S2_beta_err_prime=beta_error(omega_k)[1][index_2[0]]
S4_beta_err_prime=beta_error(omega_k)[1][index_4[0]]

sigma_C0_prime=np.sqrt(C0_inten_err_prime**2 + C0_beta_err_prime**2  )
sigma_C2_prime=np.sqrt(C2_inten_err_prime**2 + C2_beta_err_prime**2  )
sigma_C4_prime=np.sqrt(C4_inten_err_prime**2 + C4_beta_err_prime**2  )
sigma_S2_prime=np.sqrt(S2_inten_err_prime**2 + S2_beta_err_prime**2  )
sigma_S4_prime=np.sqrt(S4_inten_err_prime**2 + S4_beta_err_prime**2  )



[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111.
 112. 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125.
 126. 127. 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139.
 140. 141. 142. 143. 144. 145. 146. 147. 148. 149. 150. 151. 152. 153.
 154. 155. 156. 157. 158. 159. 160. 161. 162. 163. 164. 165. 166. 167.
 168. 169. 170. 171. 172. 173. 174. 175. 176. 177. 178. 179.]


In [475]:
print("C0_prime=", Cs_prime[0])
print("C2_prime=", Cs_prime[2]) 
print("C4_prime=", Cs_prime[4])
print("S2_prime=", Ss_prime[2])
print("S4_prime=", Ss_prime[4])

print("C0_inten_err_prime=", Cs_inten_err_prime[0])
print("C2_inten_err_prime=", Cs_inten_err_prime[2]) 
print("C4_inten_err_prime=", Cs_inten_err_prime[4])
print("S2_inten_err_prime=", Ss_inten_err_prime[2])
print("S4_inten_err_prime=", Ss_inten_err_prime[4])

print("C0_beta_err_prime=", Cs_beta_err_prime[0])
print("C2_beta_err_prime=", Cs_beta_err_prime[2]) 
print("C4_beta_err_prime=", Cs_beta_err_prime[4])
print("S2_beta_err_prime=", Ss_beta_err_prime[2])
print("S4_beta_err_prime=", Ss_beta_err_prime[4])

print("sigma_C0_prime=", sigma_C0_prime)
print("sigma_C2_prime=", sigma_C2_prime)
print("sigma_C4_prime=", sigma_C4_prime)
print("sigma_S2_prime=", sigma_S2_prime)
print("sigma_S4_prime=", sigma_S4_prime)

C0_prime= 7.491388270055915e-06
C2_prime= -7.674339670164691e-08
C4_prime= 1.981485896574493e-06
S2_prime= -4.2385269657581047e-07
S4_prime= -3.056060238946258e-06
C0_inten_err_prime= 1.4124840215418762e-08
C2_inten_err_prime= 9.987770299499064e-09
C4_inten_err_prime= 9.987770299499062e-09
S2_inten_err_prime= 9.98777029949906e-09
S4_inten_err_prime= 9.987770299499062e-09
C0_beta_err_prime= 0.0
C2_beta_err_prime= 2.725605818705859e-09
C4_beta_err_prime= 3.165261720313335e-09
S2_beta_err_prime= 6.9228970640078186e-09
S4_beta_err_prime= 1.2258333498869621e-08
sigma_C0_prime= 7.062420107709381e-09
sigma_C2_prime= 1.0352993897154527e-08
sigma_C4_prime= 1.047732968430585e-08
sigma_S2_prime= 1.2152450753424333e-08
sigma_S4_prime= 1.581209333785583e-08


### Calculate calibration parameters according to formulas in paper
$\beta=\frac{\pi}{2}+1+2 \frac{\sqrt{C_{4}^2 + S_{4}^2}-C_{0}}{\sqrt{C_{4}^2 + S_{4}^2}+C_{0}}$

$\alpha_{0}=\frac{cot \tilde{\alpha}}{2}- \frac{1}{sin(2 \tilde{\alpha})}\frac{\sqrt{C_{4}^2 + S_{4}^2} - \tilde{C_{0}}}{ \sqrt{C_{4}^2 + S_{4}^2 }- C_{0} }$

$\beta_{0}= \frac{1}{4}\left(arctan\frac{S_{4}}{C_{4}} - 2\alpha_{0} \right)$

With the variables with tilde being from the measurements for $\alpha$=45°

In [476]:
delta=np.pi/2+1+2*(( np.sqrt(Cs[4]**2 + Ss[4]**2 ) -Cs[0]) /( np.sqrt( Cs[4]**2 + Ss[4]**2 )+Cs[0] ))
alpha_0=(1/np.tan(np.pi/4))/2 -(1/sin(2*np.pi/4))*(( np.sqrt(Cs[4]**2 + Ss[4]**2 ) -Cs_prime[0]) /( np.sqrt( Cs[4]**2 + Ss[4]**2 )-Cs[0] ))
beta_0=( np.arctan(Ss[4]/Cs[4]) -2*alpha_0 )/4 
print('delta',delta*180/np.pi, delta)
print('alpha_0',alpha_0*180/np.pi, alpha_0)
print('beta_0',beta_0*180/np.pi, beta_0)
print(cos(alpha_0))
beta_0=0.325747067200380
alpha_0=-0.0671078747501453
delta=1.5165869244284742

1.0684267687742498e-05
delta 86.89403003447751 1.5165869244284742
alpha_0 -3.84499799527587 -0.0671078747501453
beta_0 18.6639321393462 0.325747067200380
0.997749111496985


### Calculate errors on parameters



In [477]:
# Error on retardance beta

dbdC4=4*C4*C0/( ((C4**2 + S4**2)**(1/2))*((C4**2 + S4**2)**(1/2) + C0)**2 )
dbdS4=4*S4*C0/( ((C4**2 + S4**2)**(1/2))*((C4**2 + S4**2)**(1/2) + C0)**2 )
dbdC0=-4*((C4**2 + S4**2)**(1/2))/((C4**2 + S4**2)**(1/2) + C0)**2

sigma_delta=np.sqrt( (dbdC4*sigma_C4)**2 +(dbdS4*sigma_S4)**2 + (dbdC0*sigma_C0)**2 )

print("sigma_delta=",sigma_delta*180/np.pi)
print(dbdC4*sigma_C4)
print(dbdS4*sigma_S4)
print(dbdC0*sigma_C0)
# Error on alpha_0
alpha_tilde=45*np.pi/180
sigma_alpha_tilde=1*np.pi/180

dalpha_0dalpha_tilde=-1/(2*np.sin(alpha_tilde)**2) + ( (((C4**2 + S4**2)**(1/2)) -C0_prime)/(((C4**2 + S4**2)**(1/2)) -C0) )*(2*np.cos(2*alpha_tilde))/(np.sin(2*alpha_tilde)**2)
dalpha_0dS4=-S4*(C0_prime-C0)/(  (sin(2*alpha_tilde))* (((C4**2 + S4**2)**(1/2)))* (C0- ((C4**2 + S4**2)**(1/2))) )
dalpha_0dC4=-C4*(C0_prime-C0)/(  (sin(2*alpha_tilde))* (((C4**2 + S4**2)**(1/2)))* (C0- ((C4**2 + S4**2)**(1/2))) )
dalpha_0dC0_prime=1/(np.sin(2*alpha_tilde)*(((C4**2 + S4**2)**(1/2))-C0))
dalpha_0dC0=-(((C4**2 + S4**2)**(1/2))- C0_prime)/( (sin(2*alpha_tilde))*(((C4**2 + S4**2)**(1/2))-C0)**2 )

sigma_alpha_0= ((dalpha_0dalpha_tilde*sigma_alpha_tilde)**2 + (dalpha_0dS4*sigma_S4)**2 + (dalpha_0dC4*sigma_C4)**2 +(dalpha_0dC0_prime*sigma_C0_prime)**2 +(dalpha_0dC0*sigma_C0)**2)**(0.5)

print("sigma_alpha_0=",sigma_alpha_0*180/np.pi)
print(dalpha_0dalpha_tilde*sigma_alpha_tilde)
print(dalpha_0dS4*sigma_S4)
print(dalpha_0dC4*sigma_C4)
print(dalpha_0dC0_prime*sigma_C0_prime)
print(dalpha_0dC0*sigma_C0)
# Error on beta_0

dbeta_0dS4=C4/(4*(S4**2 +C4**2))
dbeta_0dC4=-S4/(4*(S4**2 +C4**2))
dbeta_0dalpha_0=-0.5

sigma_beta_0=( (dbeta_0dS4*sigma_S4)**2 + (dbeta_0dC4*sigma_C4)**2 + (dbeta_0dalpha_0*sigma_alpha_0)**2)**(0.5)

print("sigma_beta_0=", sigma_beta_0*180/np.pi)
print(dbeta_0dalpha_0*sigma_alpha_0)
print(dbeta_0dC4*sigma_C4)
print(dbeta_0dS4*sigma_S4)

sigma_delta= 0.23457261141042854
-0.0009301521417275405
-0.003958321786374016
-0.0004773562336277164
sigma_alpha_0= 1.00198696039207
-0.01745329251994329
-7.85051078416677e-9
-1.84476397161156e-9
-0.0009575263108585474
0.000543020711168338
sigma_beta_0= 0.503975884984942
-0.00874398576044583
0.0007573323465674836
-0.0005826274960510324


### Calculate the other Stokes parameters
\begin{align}
M=\frac{2}{1-cos\delta}\left[C_{4}cos(2\alpha + 4\beta_{0}) + S_{4}sin(2\alpha + 4\beta_{0})\right]\\
|M|=\frac{2}{1-cos\delta} \left( C_{4}^2 + S_{4}^2  \right)^{\frac{1}{2}}\\
C=\frac{2}{1-cos\delta}\left[S_{4}cos(2\alpha + 4\beta_{0}) - C_{4}sin(2\alpha + 4\beta_{0})\right]\\
S=\frac{C_{2}}{sin\delta sin(2\alpha + 4\beta_{0})}=\frac{-S_{2}}{sin \delta cos(2\alpha + 4\beta_{0})}\\
|S|= \frac{  \left( C_{2}^2 + S_{2}^2\right)^{\frac{1}{2}}}{  sin^2 \delta}\\
I=C_{0} - \frac{1+cos \delta}{1-cos \delta} \left[  C_{4} cos(4 \alpha+ 4\beta_{0}) + S_{4} sin(4\alpha +4\beta_{0})     \right]
\end{align} 
In order to obtain the values of the parameters we need to measure the angles $\alpha$ and $\beta_{0}$, where $\alpha$ can be obtained while setting the system up and $\beta_{0}$ can be obtained by measuring either strongly circularly polarized light or strongly linearly polarized light and using the relations above.

For strongly linearly polarized light with C=S=0 we get
\begin{align}
tan(4\beta_{0} + 2\alpha)=\frac{S_{4}}{C_{4}}
\end{align}



The retardance from the previous calculations was used

For $\beta_{0}$ from the formula for horizontally polarized light we get

In [478]:
display('beta_{0}', beta_0)
print(beta_0*180/np.pi)
print('Ss4/Cs4', Ss[4]/Cs[4])

'beta_{0}'

18.663932139346183
Ss4/Cs4 2.3519404321231256


In [479]:
modulus_S=sqrt( Cs[2]**2 + Ss[2]**2  )/(np.sin(delta)**2)
modulus_M=sqrt( Cs[4]**2 + Ss[4]**2  )*2/(1-np.cos(delta))

print('|S|=', modulus_S)
print('|M|=', modulus_M)

# Calculate Stokes parameters from angles




# Calculate Stokes parameters from angles and Fourier coefficients
M=(2/(1-np.cos(delta)))*( Cs[4]*cos(2*alpha_0-4*beta_0 ) + Ss[4]*sin(2*alpha_0-4*beta_0)  )
C=(2/(1-np.cos(delta)))*( Ss[4]*cos(2*alpha_0-4*beta_0 ) - Cs[4]*sin(2*alpha_0-4*beta_0)  )
# Calculate S from Cs[2] and Ss[2] each to check on plausibility (see Formula)
S_first=Cs[2]/(sin(delta)*sin(2*alpha_0-2*beta_0))
S_second=-Ss[2]/(sin(delta)*cos(2*alpha_0-2*beta_0))
I=Cs[0]-((1+np.cos(delta))/(1-np.cos(delta)))*(Cs[4]*cos(4*alpha_0 - 4*beta_0)+Ss[4]*sin(4*alpha_0 - 4*beta_0))
L=np.sqrt(Cs[4]**2 +Ss[4]**2)/sin(delta/2)**2
L_other=sqrt(M**2 + C**2)
# Modulation ratio
# Corresponds to expectations for vertically polarized light
#eta=M*(1-np.cos(delta))/(2*I + M*np.cos(2*alpha*(1+np.cos(delta))))

|S|= 7.43791274231166e-7
|M|= 6.99622481696474e-6


### Comparison computed Stokes parameters and measured Stokes parameters

$$\vec{S}_{Meas.}=\left(\begin{array}{c} 1 \\ 0.999 \\ 0.003 \\ 0.020 \end{array}\right)$$

In [480]:
print('I=', I)
print('M/I=', (M/I) )
print('C/I=', (C/I)   )
print('S from C_{2}/I=', S_first/I)
print('S from -S_{2}/I=', S_second/I)
print('L/I',L/I)
print('L_other/I', L_other/I)

I= 7.28973130480259e-6
M/I= 0.825329998732581
C/I= -0.489821899060812
S from C_{2}/I= 0.105763117972208
S from -S_{2}/I= 0.0978437098696281
L/I 0.959736995018667
L_other/I 0.959736995018667


We see that the S values have different signs, which could be due to weak 2 $\omega$ signals. The other values correspond to the measured values.

The magnitude of S can always be computed without knowing the angles $\alpha$ and $\beta$

In [481]:
print('|S|=', modulus_S)
print('Normalized |S| with sign from S_{2}=', (modulus_S/I)*np.sign(S_second) )
print('Normalized |S| with sign from C_{2}=', (modulus_S/I)*np.sign(S_first) )

|S|= 7.43791274231166e-7
Normalized |S| with sign from S_{2}= 0.102032742104108
Normalized |S| with sign from C_{2}= 0.102032742104108


For horizontally polarized light we can also calculate vertical polarization without knowing the angles $\alpha$ and $\beta$
$$|M|=\frac{2}{1-cos\delta} \left( C_{4}^2 + S_{4}^2  \right)^{\frac{1}{2}}$$

In [482]:
display('|M|', modulus_M)
display('|M|/I', modulus_M/I)

'|M|'

'|M|/I'

These values are also in agreement with the measurement

Absolute values for completeness

In [483]:
print('M=', M)
print('C=', C)
print('S_first=', S_first)
print('S_second=', S_second)
print('I=', I)
#print('Modulation ratio eta=', eta)

M= 6.01643392855357e-6
C= -3.57067003136145e-6
S_first= 7.70984711975535e-7
S_second= 7.13254354814649e-7
I= 7.28973130480259e-6


### Evaluate Measurement

Now where we have obtained the parameters from the calibration for $\alpha_{0}$, $\beta_{0}$ and $\delta$ we can plug them into our calculations and evalutate any measurement

In [484]:
# Calculate discrete Fourier coefficients for an uneven number of data points N=2L+1
names=['H-light', 'L-light', 'E-light']

# Choose the measurement
z=2
intensity=power[z]

L=180
N=2*L
index=np.linspace(0,359,360)*(np.pi/180)


omega_k=[]
# Define step size
delta_beta=(index[1]-index[0])
#print(index)
for i in np.arange(L):
    omega_k.append(2*np.pi*i/(N*delta_beta))
omega_k=np.array(omega_k)
# Calculate Fourier series coefficients
C_omega_k=[]
S_omega_k=[]
#print(len(index))
print(omega_k)
# Sum in definition of C_omega_k with cos terms

def sum_cos(x):
    ''' Calculate formula from above with variable x=omega_{k}'''
    sum_cos=0
    j=0
    for i in index:
        sum_cos=sum_cos+ intensity[j]*np.cos(x*i)
        j+=1
    return sum_cos
a=sum_cos(omega_k)

# Sum in definition of S_omega_k with sin terms
def sum_sin(x):
    '''Same with sin'''
    sum_sin=0
    j=0
    for i in index:
        sum_sin+= intensity[j]*np.sin(x*i)
        j+=1
    return sum_sin

#print(sum_sin(omega_k))
def fourier_coefficients(x):
    '''Calculate complete Fourier coefficients'''
    C_omega_k=(2/N)* sum_cos(x)
    S_omega_k=(2/N)* sum_sin(x)
    return C_omega_k, S_omega_k

sigma_I=0.134*10**(-6)

def sum_cos_inten_err(x):
    ''' Calculate formula from above with variable x=omega_{k}'''
    sum_cos_error=0
    for i in index:
        sum_cos_error=sum_cos_error+ (np.cos(x*i))**2
        
    return sum_cos_error

def sum_sin_inten_err(x):
    ''' Same with sin'''
    sum_sin_error=0
    for i in index:
        sum_sin_error=sum_sin_error+ (np.sin(x*i))**2
        
    return sum_sin_error

def intensity_error(x):
    C_omega_k_inten_error=np.sqrt(sum_cos_inten_err(x)*(2*sigma_I/N)**2 )
    S_omega_k_inten_error=np.sqrt(sum_sin_inten_err(x)*(2*sigma_I/N)**2 )
    return C_omega_k_inten_error, S_omega_k_inten_error

# Define error on sigma_beta
sigma_beta=0.3*np.pi/180

def sum_cos_beta_err(x):
    
    sum_cos=0
    j=0
    for i in index:
        sum_cos=sum_cos+ (intensity[j]*(np.sin(x*i)))**2
        j+=1
    return sum_cos

def sum_sin_beta_err(x):
    
    sum_sin=0
    j=0
    for i in index:
        sum_sin=sum_sin+ (intensity[j]*(x*np.cos(x*i)))**2
        j+=1
    return sum_sin

def beta_error(x):
    C_omega_k_beta_error=np.sqrt(sum_cos_beta_err(x)*(2*sigma_beta/N)**2)
    S_omega_k_beta_error=np.sqrt(sum_sin_beta_err(x)*(2*sigma_beta/N)**2)
    return C_omega_k_beta_error, S_omega_k_beta_error

# Loop to get all Fourier coefficients
Cs=[]
indices=[]
Ss=[]

Cs_inten_err=[]
Ss_inten_err=[]

Cs_beta_err=[]
Ss_beta_err=[]

#Cs_omega_err=[]
#Ss_omega_err=[]

# Calculate only first 11 Fourier coefficients

# Find indices, where omega_k is closest to desired integer values

for i in np.arange(11):
    indices.append(np.where(  np.absolute(omega_k-i)==np.amin(np.absolute(omega_k-i)) )[0][0] )

for i in np.arange(11):
    Cs.append(fourier_coefficients(omega_k)[0][indices[i]])
    Ss.append(fourier_coefficients(omega_k)[1][indices[i]])
    Cs_inten_err.append(intensity_error(omega_k)[0][indices[i]])
    Ss_inten_err.append(intensity_error(omega_k)[1][indices[i]])
    Cs_beta_err.append(beta_error(omega_k)[0][indices[i]])
    Ss_beta_err.append(beta_error(omega_k)[1][indices[i]])
    #Cs_omega_err.append(omega_error(omega_k)[0][indices[i]])
    #Ss_omega_err.append(omega_error(omega_k)[1][indices[i]])
Cs[0]=Cs[0]*0.5
#Calculating Fourier coefficients, which correspond to omega_k=2, 4
# Same method as above, did it just to check if the method is right
# Find index of entry of omega_k, which is closest to omega_k = 0,2 or 4
index_0,=np.where( omega_k==np.amin(omega_k) )
index_2,=np.where(  np.absolute(omega_k-2)==np.amin(np.absolute(omega_k-2)) )
index_4,=np.where(  np.absolute(omega_k-4)==np.amin(np.absolute(omega_k-4)) )


#print('Cs=', Cs)
#print(omega_k)
# Calculate Fourier coefficients for omega_k = 2,4
# Factor 0.5 for omega_k=0 to account for special case of fourier series coefficient formula
C0=fourier_coefficients(omega_k)[0][index_0[0]]*0.5
C2=fourier_coefficients(omega_k)[0][index_2[0]]
C4=fourier_coefficients(omega_k)[0][index_4[0]]
S2=fourier_coefficients(omega_k)[1][index_2[0]]
S4=fourier_coefficients(omega_k)[1][index_4[0]]

C0_inten_err=intensity_error(omega_k)[0][index_0[0]]*0.5
C2_inten_err=intensity_error(omega_k)[0][index_2[0]]
C4_inten_err=intensity_error(omega_k)[0][index_4[0]]
S2_inten_err=intensity_error(omega_k)[1][index_2[0]]
S4_inten_err=intensity_error(omega_k)[1][index_4[0]]

C0_beta_err=beta_error(omega_k)[0][index_0[0]]*0.5
C2_beta_err=beta_error(omega_k)[0][index_2[0]]
C4_beta_err=beta_error(omega_k)[0][index_4[0]]
S2_beta_err=beta_error(omega_k)[1][index_2[0]]
S4_beta_err=beta_error(omega_k)[1][index_4[0]]


sigma_C0=np.sqrt(C0_inten_err**2 + C0_beta_err**2  )
sigma_C2=np.sqrt(C2_inten_err**2 + C2_beta_err**2  )
sigma_C4=np.sqrt(C4_inten_err**2 + C4_beta_err**2  )
sigma_S2=np.sqrt(S2_inten_err**2 + S2_beta_err**2  )
sigma_S4=np.sqrt(S4_inten_err**2 + S4_beta_err**2  )

[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111.
 112. 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125.
 126. 127. 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139.
 140. 141. 142. 143. 144. 145. 146. 147. 148. 149. 150. 151. 152. 153.
 154. 155. 156. 157. 158. 159. 160. 161. 162. 163. 164. 165. 166. 167.
 168. 169. 170. 171. 172. 173. 174. 175. 176. 177. 178. 179.]


In [485]:
print("C0=", Cs[0])
print("C2=", Cs[2]) 
print("C4=", Cs[4])
print("S2=", Ss[2])
print("S4=", Ss[4])

print("C0_inten_err=", Cs_inten_err[0])
print("C2_inten_err=", Cs_inten_err[2]) 
print("C4_inten_err=", Cs_inten_err[4])
print("S2_inten_err=", Ss_inten_err[2])
print("S4_inten_err=", Ss_inten_err[4])

print("C0_beta_err=", Cs_beta_err[0])
print("C2_beta_err=", Cs_beta_err[2]) 
print("C4_beta_err=", Cs_beta_err[4])
print("S2_beta_err=", Ss_beta_err[2])
print("S4_beta_err=", Ss_beta_err[4])


print("sigma_C0=", sigma_C0)
print("sigma_C2=", sigma_C2)
print("sigma_C4=", sigma_C4)
print("sigma_S2=", sigma_S2)
print("sigma_S4=", sigma_S4)


C0= 8.053885018297375e-06
C2= -1.5427208153585117e-07
C4= -7.019852955492545e-08
S2= -4.045991192126805e-07
S4= -2.6091879703371646e-06
C0_inten_err= 1.4124840215418762e-08
C2_inten_err= 9.987770299499064e-09
C4_inten_err= 9.987770299499062e-09
S2_inten_err= 9.98777029949906e-09
S4_inten_err= 9.987770299499062e-09
C0_beta_err= 0.0
C2_beta_err= 3.2459985523074978e-09
C4_beta_err= 3.28511602615496e-09
S2_beta_err= 6.439733374457562e-09
S4_beta_err= 1.271978665217802e-08
sigma_C0= 7.062420107709381e-09
sigma_C2= 1.0502002768859756e-08
sigma_C4= 1.0514159160905628e-08
sigma_S2= 1.1883842875503619e-08
sigma_S4= 1.6172462027547992e-08


In [486]:
modulus_S=sqrt( Cs[2]**2 + Ss[2]**2  )/(np.sin(delta)**2)
modulus_M=sqrt( Cs[4]**2 + Ss[4]**2  )*2/(1-np.cos(delta))

print('|S|=', modulus_S)
print('|M|=', modulus_M)

# Calculate Stokes parameters from angles




# Calculate Stokes parameters from angles and Fourier coefficients
M=(2/(1-np.cos(delta)))*( Cs[4]*cos(2*alpha_0-4*beta_0 ) + Ss[4]*sin(2*alpha_0-4*beta_0)  )
C=(2/(1-np.cos(delta)))*( Ss[4]*cos(2*alpha_0-4*beta_0 ) - Cs[4]*sin(2*alpha_0-4*beta_0)  )
# Calculate S from Cs[2] and Ss[2] each to check on plausibility (see Formula)
S_first=Cs[2]/(sin(delta)*sin(2*alpha_0-2*beta_0))
S_second=-Ss[2]/(sin(delta)*cos(2*alpha_0-2*beta_0))
I=Cs[0]-((1+np.cos(delta))/(1-np.cos(delta)))*(Cs[4]*cos(4*alpha_0 - 4*beta_0)+Ss[4]*sin(4*alpha_0 - 4*beta_0))
L=np.sqrt(Cs[4]**2 +Ss[4]**2)/sin(delta/2)**2
L_other=sqrt(M**2 + C**2)
# Modulation ratio
# Corresponds to expectations for vertically polarized light
#eta=M*(1-np.cos(delta))/(2*I + M*np.cos(2*alpha*(1+np.cos(delta))))

|S|= 4.34288049154608e-7
|M|= 5.51931657966108e-6


In [ ]:
### Print all calculated values

In [487]:
print("C0=", Cs[0])
print("C2=", Cs[2]) 
print("C4=", Cs[4])
print("S2=", Ss[2])
print("S4=", Ss[4])
print('I=', I)
print('M/I=', (M/I) )
print('C/I=', (C/I)   )
print('S from C_{2}/I=', S_first/I)
print('S from -S_{2}/I=', S_second/I)
print('L/I',L/I)
print('L_other/I', L_other/I)

print('|S|=', modulus_S)
print('|S|/I with sign from S_{2}=', (modulus_S/I)*np.sign(S_second) )
print('|S|/I with sign from C_{2}=', (modulus_S/I)*np.sign(S_first) )

print('(L/I)**2 + (|S|/I)**2=',(L/I)**2 + (modulus_S/I)**2)

print('|M|', modulus_M)
print('|M|/I', modulus_M/I)

C0= 8.053885018297375e-06
C2= -1.5427208153585117e-07
C4= -7.019852955492545e-08
S2= -4.045991192126805e-07
S4= -2.6091879703371646e-06
I= 5.14570467498417e-6
M/I= 1.05882256657292
C/I= -0.171404786026824
S from C_{2}/I= 0.0424483269849296
S from -S_{2}/I= 0.111395828723233
L/I 1.07260655794982
L_other/I 1.07260655794982
|S|= 4.34288049154608e-7
|S|/I with sign from S_{2}= 0.0843981682947912
|S|/I with sign from C_{2}= 0.0843981682947912
(L/I)**2 + (|S|/I)**2= 1.15760787896848
|M| 5.51931657966108e-6
|M|/I 1.07260655794982


In [488]:
print('M=', M)
print('C=', C)
print('S_first=', S_first)
print('S_second=', S_second)
print('I=', I)
#print('Modulation ratio eta=', eta)

M= 5.44838823079301e-6
C= -8.81998408772888e-7
S_first= 2.18426554611609e-7
S_second= 5.73210036634875e-7
I= 5.14570467498417e-6


### Calculate Errors on Stokes parameters



In [499]:
# Error on intensity



dIdC0=1
dIddelta=(2*np.sin(delta))*(C4*np.cos(4*alpha_0 + 4*alpha_tilde - 4*beta_0) + S4*np.sin(4*alpha_0 + 4*alpha_tilde - 4*beta_0))/((1-np.cos(delta))**2)
dIdC4=(np.cos(delta)+1)*(np.cos(4*alpha_0 + 4*alpha_tilde - 4*beta_0))/(np.cos(delta)-1)
dIdS4=(np.cos(delta)+1)*(np.sin(4*alpha_0 + 4*alpha_tilde - 4*beta_0))/(np.cos(delta)-1)
dIdalpha_0=4*(np.cos(delta)+1)*((S4*np.cos(4*alpha_0 + 4*alpha_tilde - 4*beta_0) - C4*np.sin(4*alpha_0 + 4*alpha_tilde - 4*beta_0)))/(np.cos(delta)-1)
dIdalpha_tilde=4*(np.cos(delta)+1)*((S4*np.cos(4*alpha_0 + 4*alpha_tilde - 4*beta_0) - C4*np.sin(4*alpha_0 + 4*alpha_tilde - 4*beta_0)))/(np.cos(delta)-1)
dIdbeta_0=-4*(np.cos(delta)+1)*((S4*np.cos(4*alpha_0 + 4*alpha_tilde - 4*beta_0) - C4*np.sin(4*alpha_0 + 4*alpha_tilde - 4*beta_0)))/(np.cos(delta)-1)

sigma_I= ((dIdC0*sigma_C0)**2 + (dIddelta*sigma_delta)**2 + (dIdC4*sigma_C4)**2 + (dIdS4*sigma_S4)**2 + (dIdalpha_0*sigma_alpha_0)**2 + (dIdalpha_tilde*sigma_alpha_tilde)**2+ (dIdbeta_0*sigma_beta_0)**2)**(1/2)

print("sigma_I",sigma_I)

# Error on M

dMddelta=-(2*np.sin(delta))*(C4*np.cos(2*alpha_0 +2*alpha_tilde-4*beta_0) + S4*np.sin(2*alpha_0 +2*alpha_tilde -4*beta_0))/(1-np.cos(delta))**2
dMdC4=-2*np.cos(2*alpha_0 +2*alpha_tilde-4*beta_0)/(np.cos(delta)-1)
dMdS4=-2*np.sin(2*alpha_0 +2*alpha_tilde-4*beta_0)/(np.cos(delta)-1)
dMdalpha_0=4*((C4*np.sin(2*alpha_0 +2*alpha_tilde-4*beta_0) - S4*np.cos(2*alpha_0 +2*alpha_tilde -4*beta_0)))/(np.cos(delta)-1)
dMdalpha_tilde=4*((C4*np.sin(2*alpha_0 +2*alpha_tilde-4*beta_0) - S4*np.cos(2*alpha_0 +2*alpha_tilde -4*beta_0)))/(np.cos(delta)-1)
dMdbeta_0=8*((S4*np.cos(2*alpha_0 +2*alpha_tilde-4*beta_0) - C4*np.sin(2*alpha_0 +2*alpha_tilde -4*beta_0)))/(np.cos(delta)-1)

sigma_M=( (dMddelta*sigma_delta)**2 + (dMdC4*sigma_C4)**2 +(dMdS4*sigma_S4)**2 + (dMdalpha_0*sigma_alpha_0)**2 + (dMdalpha_tilde*sigma_alpha_tilde)**2 + (dMdbeta_0*sigma_beta_0)**2)**(0.5)

print("sigma_M",sigma_M)
print(dMddelta*sigma_delta)
print(dMdC4*sigma_C4)
print(dMdS4*sigma_S4)
print(dMdalpha_0*sigma_alpha_0)
print(dMdalpha_tilde*sigma_alpha_tilde)
print(dMdbeta_0*sigma_beta_0)
sigma_M_over_I=((sigma_M/I)**2 + (-M*sigma_I/I**2)**2)**(0.5)

print("sigma_M_over_I",sigma_M_over_I)

# Error on C

dCddelta=-(2*np.sin(delta))*(S4*np.cos(2*alpha_0 +2*alpha_tilde-4*beta_0) - C4*np.sin(2*alpha_0 +2*alpha_tilde -4*beta_0))/(1-np.cos(delta))**2
dCdC4=2*np.sin(2*alpha_0 +2*alpha_tilde-4*beta_0)/(np.cos(delta)-1)
dCdS4=-2*np.cos(2*alpha_0 +2*alpha_tilde-4*beta_0)/(np.cos(delta)-1)
dCdalpha_0=4*((C4*np.cos(2*alpha_0 +2*alpha_tilde-4*beta_0) +S4*np.sin(2*alpha_0 +2*alpha_tilde -4*beta_0)))/(np.cos(delta)-1)
dCdalpha_tilde=4*((C4*np.cos(2*alpha_0 +2*alpha_tilde-4*beta_0) +S4*np.sin(2*alpha_0 +2*alpha_tilde -4*beta_0)))/(np.cos(delta)-1)
dCdbeta_0=-8*((C4*np.cos(2*alpha_0 +2*alpha_tilde-4*beta_0) + S4*np.sin(2*alpha_0 +2*alpha_tilde -4*beta_0)))/(np.cos(delta)-1)

sigma_C=( (dCddelta*sigma_delta)**2 + (dCdC4*sigma_C4)**2 +(dCdS4*sigma_S4)**2 + (dCdalpha_0*sigma_alpha_0)**2 + (dCdalpha_tilde*sigma_alpha_tilde)**2 + (dCdbeta_0*sigma_beta_0)**2)**(0.5)

print("sigma_C",sigma_C)
print(dCddelta*sigma_delta)
print(dCdC4*sigma_C4)
print(dCdS4*sigma_S4)
print(dCdalpha_0*sigma_alpha_0)
print(dCdalpha_tilde*sigma_alpha_tilde)
print(dCdbeta_0*sigma_beta_0)

sigma_C_over_I=((sigma_C/I)**2 + (-C*sigma_I/I**2)**2)**(0.5)

print("sigma_C_over_I",sigma_C_over_I)

# Error on S from C2

dS_C2dC2=1/((np.sin(delta))*(np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)))
dS_C2ddelta=-C2*np.cos(delta)/(np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)*sin(delta)**2)
dS_C2dalpha_0=-2*C2*np.cos(2*alpha_0 + 2*alpha_tilde - 2*beta_0)/(np.sin(delta)*np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)**2)
dS_C2dalpha_tilde=-2*C2*np.cos(2*alpha_0 + 2*alpha_tilde - 2*beta_0)/(np.sin(delta)*np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)**2)
dS_C2dbeta_0=2*C2*np.cos(2*alpha_0 + 2*alpha_tilde - 2*beta_0)/(np.sin(delta)*np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)**2)

sigma_S_C2=( (dS_C2ddelta*sigma_delta)**2 + (dS_C2dC2*sigma_C2)**2  + (dS_C2dalpha_0*sigma_alpha_0)**2 + (dS_C2dalpha_tilde*sigma_alpha_tilde)**2 + (dS_C2dbeta_0*sigma_beta_0)**2)**(0.5)

print("sigma_S_C2",sigma_S_C2)
print(dS_C2ddelta*sigma_delta)
print(dS_C2dC2*sigma_C2)
print(dS_C2dalpha_0*sigma_alpha_0)
print(dS_C2dalpha_tilde*sigma_alpha_tilde)
print(dS_C2dbeta_0*sigma_beta_0)
sigma_S_C2_over_I=((sigma_S_C2/I)**2 + (-S_first*sigma_I/I**2)**2)**(0.5)

print("sigma_S_C2_over_I",sigma_S_C2_over_I)


# Error on S from S2

dS_S2dS2=-1/((np.sin(delta))*(np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)))
dS_S2ddelta=S2*np.cos(delta)/(np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)*sin(delta)**2)
dS_S2dalpha_0=2*S2*np.cos(2*alpha_0 + 2*alpha_tilde - 2*beta_0)/(np.sin(delta)*np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)**2)
dS_S2dalpha_tilde=2*S2*np.cos(2*alpha_0 + 2*alpha_tilde - 2*beta_0)/(np.sin(delta)*np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)**2)
dS_S2dbeta_0=-2*S2*np.cos(2*alpha_0 + 2*alpha_tilde - 2*beta_0)/(np.sin(delta)*np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)**2)

sigma_S_S2=( (dS_S2ddelta*sigma_delta)**2 + (dS_S2dS2*sigma_S2)**2  + (dS_S2dalpha_0*sigma_alpha_0)**2 + (dS_S2dalpha_tilde*sigma_alpha_tilde)**2 + (dS_S2dbeta_0*sigma_beta_0)**2)**(0.5)

print("sigma_S_S2",sigma_S_S2)

sigma_S_S2_over_I=((sigma_S_S2/I)**2 + (-S_second*sigma_I/I**2)**2)**(0.5)

print("sigma_S_S2_over_I",sigma_S_S2_over_I)

# Error on |M|

dmodMdC4=-2*C4/(((C4**2 + S4**2)**(0.5))*(np.cos(delta)-1))
dmodMdS4=-2*S4/(((C4**2 + S4**2)**(0.5))*(np.cos(delta)-1))
dmodMddelta=(-2*np.sin(delta)*(C4**2 + S4**2)**(0.5))/(1-np.cos(delta))**2

sigma_modM=((dmodMdC4*sigma_C4)**2 + (dmodMdS4*sigma_S4)**2 + (dmodMddelta*sigma_delta)**2)**(0.5)

print("sigma_modM",sigma_modM)

sigma_modM_over_I=((sigma_modM/I)**2 + (-modulus_M*sigma_I/I**2)**2)**(0.5)

print("sigma_modM_over_I",sigma_modM_over_I)

# Error on |S|

dmodSdC2=C2/((np.sin(delta)**2)*(C2**2 + S2**2)**(0.5))
dmodSdS2=S2/((np.sin(delta)**2)*(C2**2 + S2**2)**(0.5))
dmodSddelta=(-2*np.cos(delta)*(C2**2 + S2**2)**(0.5))/(np.sin(delta)**3)

sigma_modS=((dmodSdC2*sigma_C2)**2 + (dmodSdS2*sigma_S2)**2 + (dmodSddelta*sigma_delta)**2)**(0.5)


print("sigma_modS=",sigma_modS)
print(dmodSdC2*sigma_C2)
print(dmodSdS2*sigma_S2)
print(dmodSddelta*sigma_delta)
sigma_modS_over_I=((sigma_modS/I)**2 + (-modulus_S*sigma_I/I**2)**2)**(0.5)

print("sigma_modS_over_I",sigma_modS_over_I)
print(sigma_modS/I)
print((-modulus_S*sigma_I/I**2)**2)
# Error on |L|

dLdC4=C4/((np.sin(delta/2)**2)*(C4**2 + S4**2)**(0.5))
dLdS4=S4/((np.sin(delta/2)**2)*(C4**2 + S4**2)**(0.5))
dLddelta=(-(C4**2 + S4**2)**(0.5))*np.cos(delta/2)/(np.sin(delta/2)**3)

sigma_L=np.sqrt((dLdC4*sigma_C4)**2 + (dLdS4*sigma_S4)**2 + (dLddelta*sigma_delta)**2)

print('sigma_L=', sigma_L)
print(dLdC4*sigma_C4)
print(dLdS4*sigma_S4)
print(dLddelta*sigma_delta)

sigma_L_over_I=((sigma_L/I)**2 + (-L*sigma_I/(I)**2)**2)**(0.5)

print("sigma_L_over_I=", sigma_L_over_I)
print(sigma_L/I)
print(-L*sigma_I/(I)**2)

sigma_I 3.17456137803609e-8
sigma_M 3.31289593491499e-7
3.812210344181858e-09
2.2034863882899688e-08
4.5549942912064596e-09
-1.90562516429739e-7
-1.9018462710849333e-07
1.91696931515068e-7
sigma_M_over_I 0.0647123088702804
sigma_C 6.76358456353249e-8
2.3549251071150313e-08
-2.961326165007075e-09
3.389315245991476e-08
3.08487261081840e-8
3.078755246087545e-08
-3.10323679959801e-8
sigma_C_over_I 0.0131866045704951
sigma_S_C2 1.87873666484037e-8
4.85546903232797e-11
1.4878562769963123e-08
7.64920644777346e-9
7.634037916801207e-09
-3.84737101512849e-9
sigma_S_C2_over_I 0.00366045724733728
sigma_S_S2 3.44758922533465e-8
sigma_S_S2_over_I 0.00673509023840142
sigma_modM 4.1690593087358967e-08
sigma_modM_over_I 0.0104609299377668
sigma_modS= 1.1753560923626409e-08
-3.752626382017123e-09
-1.1136730077481625e-08
-1.9295803611894237e-10
sigma_modS_over_I 0.00234274406723982
0.00228414992037268
2.71108905848852e-7
sigma_L= 4.1690593087358967e-08
-5.979472656890817e-10
-3.4185491620318066e-08
-2.38

In [490]:
plt.plot(angle[z], power[z], label='measurement')

def I_T(x):
    return Cs[0]+Cs[2]*np.cos(2*x*np.pi/180)+Cs[4]*np.cos(4*x*np.pi/180)+Ss[2]*np.sin(2*x*np.pi/180)+Ss[4]*np.sin(4*x*np.pi/180)
    
plt.plot(angle[z], I_T(np.array(angle[z])), '--', label='from discrete F. coeff.' )
#plt.xlabel('angle [°]')
#plt.ylabel('Intensity [W]')
plt.legend()
#plt.savefig('Test.png', fmt='png')
plt.show()

In [1]:

fig, ax=plt.subplots()
plt.plot(angle[z], power[z]*10**6, label='Measurement')

def I_T(x):
    return Cs[0]+Cs[2]*np.cos(2*x*np.pi/180)+Cs[4]*np.cos(4*x*np.pi/180)+Ss[2]*np.sin(2*x*np.pi/180)+Ss[4]*np.sin(4*x*np.pi/180)
    
plt.plot(angle[z], I_T(np.array(angle[z]))*10**(6), '--', label='From discrete FFT' )
ax.set_xlabel(r'$\Theta$ (deg)')
ax.set_ylabel(r'Intensity ($\mu W$)')
ax.legend(loc='lower left')
fig.subplots_adjust(left=.19, bottom=.22, right=.99, top=.97)

fig.set_size_inches(width, height)
plt.show()
#plt.savefig('Test.pdf', fmt='pdf')

NameError: name 'plt' is not defined

### Modulation ratio

As an additional test we can look at the modulation ratio, which depends on some of the calculated parameters, to check on the plausibility of the result.

\begin{align} \eta = \frac{I_{max} - I_{min}}{I_{max} + I_{min}} = \frac{M(1-cos\delta)}{2I + Mcos2\alpha(1+ cos\delta)} \end{align}


In [492]:
eta_intensity=(np.amax(intensity)-np.amin(intensity))/(np.amax(intensity)+np.amin(intensity))

#print('eta from intensity', eta_intensity)
#print('eta from parameters', eta)

Modulation ratios from intensity and from parameters seem to agree with each other

### Plotting the coefficients

We expect strong 0 and 4 $\omega $ signals for horizontally polarized light, which is fulfilled in this case.

In [493]:
# Produce plots like in paper
#%%
t = np.arange(11)


fig, axs = plt.subplots(1, 2, sharey=True)
# Remove horizontal space between axes

fig.subplots_adjust(wspace=0.2)

# Plot each graph, and manually set the y tick values
axs[0].stem(t, Ss)
axs[0].set_yscale('log')
axs[0].text(0.8,-0.1, 'Angular Frequency', fontsize=12, transform=axs[0].transAxes)
axs[0].text(0.6,0.7, '(Sine coeff.)', fontsize=10, transform=axs[0].transAxes)

axs[0].set_ylabel('Amplitude', fontsize=12)
axs[0].set_xticks(ticks=np.arange(0,11,2))
axs[0].set_xticks(ticks=np.arange(0,11,2))
axs[0].spines['right'].set_visible(False)

axs[1].stem(t, Cs)
axs[1].text(0.4,0.7, '(Cosine coeff.)', fontsize=10, transform=axs[1].transAxes)
#axs[1].set_yticks([])
axs[1].set_xticks(ticks=np.arange(0,11,2))
axs[1].spines['left'].set_visible(False)
plt.suptitle('Fourier series coefficients')

Text(0.5, 0.98, 'Fourier series coefficients')

### Calculate Stokes Parameter using FFT on intensity

Do a FFT on the intensity to see if the sought-after frequencies are present

In [494]:



# Number of sample points N from above
# Sample spacing i times omega

A=fftpack.rfft(intensity)




plt.figure(figsize = (12,6))
plt.title('FFT from intensity')
# Plot first 10 omegas
plt.plot(omega_k[0:10], np.abs(A)[0:10], 'o')
plt.xlabel(r'Angular Frequency [$\omega$]')
plt.ylabel('Amplitude')
plt.axvline(x=2  )
plt.axvline(x= 4  )

# Use the same formulas again

In [495]:
print(np.arctan2(-1,1)*180/np.pi )

-45.0


In [496]:
print(np.linspace(0,359,180))

[  0.           2.00558659   4.01117318   6.01675978   8.02234637
  10.02793296  12.03351955  14.03910615  16.04469274  18.05027933
  20.05586592  22.06145251  24.06703911  26.0726257   28.07821229
  30.08379888  32.08938547  34.09497207  36.10055866  38.10614525
  40.11173184  42.11731844  44.12290503  46.12849162  48.13407821
  50.1396648   52.1452514   54.15083799  56.15642458  58.16201117
  60.16759777  62.17318436  64.17877095  66.18435754  68.18994413
  70.19553073  72.20111732  74.20670391  76.2122905   78.21787709
  80.22346369  82.22905028  84.23463687  86.24022346  88.24581006
  90.25139665  92.25698324  94.26256983  96.26815642  98.27374302
 100.27932961 102.2849162  104.29050279 106.29608939 108.30167598
 110.30726257 112.31284916 114.31843575 116.32402235 118.32960894
 120.33519553 122.34078212 124.34636872 126.35195531 128.3575419
 130.36312849 132.36871508 134.37430168 136.37988827 138.38547486
 140.39106145 142.39664804 144.40223464 146.40782123 148.41340782
 150.418994